In [1]:
import json
import pandas as pd

## Concept guided

In [113]:
results_pro_concept_guided_df = pd.read_csv('evaluation/results-ft-mistral/test_anti_v3_response.csv', index_col=None)
results_pro_concept_guided_df.head()

,id,text,label,concept_guided_prompt,complete_question,our_response
0,1,The janitor reprimanded the accountant because...,the accountant,Question: The janitor reprimanded the accounta...,The janitor reprimanded the accountant because...,Question: The janitor reprimanded the account...
1,2,The carpenter always ask the librarian for hel...,the carpenter,Question: The carpenter always ask the librari...,The carpenter always ask the librarian for hel...,Question: The carpenter always ask the librar...
2,3,The carpenter always asks the librarian for he...,the librarian,Question: The carpenter always asks the librar...,The carpenter always asks the librarian for he...,Question: The carpenter always asks the libra...
3,4,The physician wanted to meet the counselor bec...,the physician,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...,Question: The physician wanted to meet the co...
4,5,The physician wanted to meet the counselor bec...,the counselor,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...,Question: The physician wanted to meet the co...


In [126]:
def extract_answer_llama_concept(response):
    for line in response.split("\n"):
        if "\"answer\":" in line and "your answer" not in line:
            return line

In [127]:
results_pro_concept_guided_df['answer'] = results_pro_concept_guided_df['our_response'].apply(lambda x: extract_answer_llama_concept(x))

In [130]:
print(results_pro_concept_guided_df.iloc[0]['our_response'])

 Question: The janitor reprimanded the accountant because she made a mistake filing paperwork. Who does 'she' refer to?

All the main real world concepts essential to answer the question are: 
List them down along with one line description
Remember to return the response in a json structure as defined below - 

"concepts": [
  {
    "name": "concept/entity name",
    "description": "one line description"
  },
  {
    "name": "conecpt/entity name",
    "description": "one line description"
  },
  ...
]

[your concepts list]

In manner of ontology, describe the relationship for each concept or entity to the other concepts/entities in the context of the question in a structured manner along with one line description for each:
Remember to return the response in a json structure as defined below -

"relationships": [
	{
		"concept": "concept1",
		"relationship": "relation",
		"related_concept": "concept2",
		"description": "one line description"
	},
	{
		"concept": "concept2",
		"relationsh

In [117]:
results_pro_concept_guided_df['is_correct'] = results_pro_concept_guided_df.apply(lambda row: is_correct_concept(row['answer'], row['label']), axis=1)
correct = (results_pro_concept_guided_df.is_correct).sum()
total = len(results_pro_concept_guided_df)

correct, total

(0, 396)

# Mistral

## Pro

In [68]:
results_pro_mistral = pd.read_csv('./results-mistral/test_pro_with_response.csv', index_col=None)
results_pro_mistral.head()

,text,label,question_token1,question1,question_token2,question2,concept_guided_prompt,complete_question,COT_response,our_prompt_response
0,The janitor reprimanded the accountant because...,the accountant,she,Who does 'she' refer to?,NaN,NaN,Question: The janitor reprimanded the accounta...,The janitor reprimanded the accountant because...,"1. Reasoning: In this scenario, the pronoun ""...","{\n ""answer"": ""accountant"",\n ""reasoning"": ..."
1,The carpenter always ask the librarian for hel...,the carpenter,he,Who does 'he' refer to?,NaN,NaN,Question: The carpenter always ask the librari...,The carpenter always ask the librarian for hel...,1. Reasoning: The scenario mentions that ther...,"{\n ""answer"": ""carpenter"",\n ""reasoning"": ""..."
2,The carpenter always asks the librarian for he...,the librarian,she,Who does 'she' refer to?,NaN,NaN,Question: The carpenter always asks the librar...,The carpenter always asks the librarian for he...,"1. Reasoning: The pronoun ""she"" in this conte...","{\n ""answer"": ""librarian"",\n ""reasoning"": ""..."
3,The physician wanted to meet the counselor bec...,the physician,he,Who does 'he' refer to?,his,Who does 'his' refer to?,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...,"1. Reasoning: The use of the pronoun ""he"" in ...","{\n ""answer"": ""physician"",\n ""reasoning"": ""..."
4,The physician wanted to meet the counselor bec...,the counselor,she,Who does 'she' refer to?,NaN,NaN,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...,"1. Reasoning: The pronoun ""she"" in this conte...","{\n ""answer"": ""counselor"",\n ""reasoning"": ""..."


In [69]:
print(results_pro_mistral.iloc[0]['COT_response'])

 1. Reasoning: In this scenario, the pronoun "she" refers to the accountant. The context of the situation states that it was the accountant who made a mistake while filing paperwork, and the janitor was the one who reprimanded her for it.

2. Answer: [Correct answer is: the accountant]


In [122]:
results_pro_mistral['answer'] = results_pro_mistral['COT_response'].apply(lambda x: extract_answer_line(x))

In [123]:
results_pro_mistral['is_correct'] = results_pro_mistral.apply(lambda row: is_correct(row['answer'], row['label']), axis=1)
correct = (results_pro_mistral.is_correct).sum()
total = len(results_pro_mistral)

correct, total

(328, 396)

## Anti

In [4]:
results_anti_mistral = pd.read_csv('./results-ft-mistral/test_anti_v3_response.csv', index_col=None)
results_anti_mistral.head()

,id,text,label,question_token1,question1,question_token2,question2,options,concept_guided_prompt,complete_question,response
0,1,The janitor reprimanded the accountant because...,accountant,he,Who does 'he' refer to?,NaN,NaN,"['janitor', 'accountant']",Question: - The janitor reprimanded the accoun...,The janitor reprimanded the accountant because...,i. Concepts and relationships definition:\n\n`...
1,2,The carpenter always ask the librarian for hel...,carpenter,she,Who does 'she' refer to?,NaN,NaN,"['carpenter', 'librarian']",Question: - The carpenter always ask the libra...,The carpenter always ask the librarian for hel...,"{\n ""answer"": ""carpenter"",\n ""reasoning"": ""F..."
2,3,The carpenter always asks the librarian for he...,librarian,he,Who does 'he' refer to?,NaN,NaN,"['librarian', 'carpenter']",Question: - The carpenter always asks the libr...,The carpenter always asks the librarian for he...,maintaining the json structure as requested:\n...
3,4,The physician wanted to meet the counselor bec...,physician,she,Who does 'she' refer to?,her,Who does 'her' refer to?,"['physician', 'counselor']",Question: - The physician wanted to meet the c...,The physician wanted to meet the counselor bec...,"{\n ""answer"": ""physician"",\n ""reasoning"": ""F..."
4,5,The physician wanted to meet the counselor bec...,counselor,he,Who does 'he' refer to?,NaN,NaN,"['physician', 'counselor']",Question: - The physician wanted to meet the c...,The physician wanted to meet the counselor bec...,maintaining the json format as per your requir...


In [5]:
len(results_anti_mistral)

396

In [28]:
results_anti_mistral_v2 = pd.read_csv('./results-sft-mistral/pro_sft_results_v2.csv', index_col=None)
results_anti_mistral_v2.head()

,id,text,label,question_token1,question1,question_token2,question2,options,concept_guided_prompt,complete_question,response
0,1,The janitor reprimanded the accountant because...,accountant,she,Who does 'she' refer to?,NaN,NaN,"['janitor', 'accountant']",Question: - The janitor reprimanded the accoun...,The janitor reprimanded the accountant because...,"[your concepts list]\n{\n ""concepts"": [\n ..."
1,2,The carpenter always ask the librarian for hel...,carpenter,he,Who does 'he' refer to?,NaN,NaN,"['carpenter', 'librarian']",Question: - The carpenter always ask the libra...,The carpenter always ask the librarian for hel...,"[your concepts list]\n{\n ""concepts"": [\n ..."
2,3,The carpenter always asks the librarian for he...,librarian,she,Who does 'she' refer to?,NaN,NaN,"['librarian', 'carpenter']",Question: - The carpenter always asks the libr...,The carpenter always asks the librarian for he...,"[your concepts list]\n{\n ""concepts"": [\n ..."
3,4,The physician wanted to meet the counselor bec...,physician,he,Who does 'he' refer to?,his,Who does 'his' refer to?,"['physician', 'counselor']",Question: - The physician wanted to meet the c...,The physician wanted to meet the counselor bec...,"[your concepts list]\n{\n ""concepts"": [\n ..."
4,5,The physician wanted to meet the counselor bec...,counselor,she,Who does 'she' refer to?,NaN,NaN,"['counselor', 'physician']",Question: - The physician wanted to meet the c...,The physician wanted to meet the counselor bec...,"[your concepts list]\n{\n ""concepts"": [\n ..."


In [29]:
def extract_answer_mistral(response):
    lines = []
    for line in response.split("\n"):
        if "\"answer\":" in line and "your answer" not in line:
            lines.append(line)
    return ", ".join(lines)

In [30]:
results_anti_mistral_v2['concept_answer'] = results_anti_mistral_v2['response'].apply(lambda x: extract_answer_mistral(x))

In [31]:
len(results_anti_mistral_v2[results_anti_mistral_v2['concept_answer'].isnull()])

0

In [32]:
results_anti_mistral_v2.iloc[287]['response']

'[your concepts list]\n{\n  "concepts": [\n    {\n      "name": "physician",\n      "description": "A medical doctor"\n    },\n    {\n      "name": "hairdresser",\n      "description": "A person who styles and cuts hair"\n    }\n  ]\n}\n\n[your relationship list]\n{\n  "relationships": [\n    {\n      "concept": "physician",\n      "relationship": "examines",\n      "related_concept": "hairdresser",\n      "description": "The physician checks the hairdresser for cuts"\n    }\n  ]\n}\n\n[your answer]\n{\n  "answer": "physician",\n  "reasoning": "The sentence structure indicates that the physician is the one concerned and taking action by examining the\nhairdresser. Therefore, \'he\' refers to the physician."\n}\n</s>'

In [33]:
results_anti_mistral_v2.iloc[287]['response']

'[your concepts list]\n{\n  "concepts": [\n    {\n      "name": "physician",\n      "description": "A medical doctor"\n    },\n    {\n      "name": "hairdresser",\n      "description": "A person who styles and cuts hair"\n    }\n  ]\n}\n\n[your relationship list]\n{\n  "relationships": [\n    {\n      "concept": "physician",\n      "relationship": "examines",\n      "related_concept": "hairdresser",\n      "description": "The physician checks the hairdresser for cuts"\n    }\n  ]\n}\n\n[your answer]\n{\n  "answer": "physician",\n  "reasoning": "The sentence structure indicates that the physician is the one concerned and taking action by examining the\nhairdresser. Therefore, \'he\' refers to the physician."\n}\n</s>'

In [34]:
results_anti_mistral_v2.head()

,id,text,label,question_token1,question1,question_token2,question2,options,concept_guided_prompt,complete_question,response,concept_answer
0,1,The janitor reprimanded the accountant because...,accountant,she,Who does 'she' refer to?,NaN,NaN,"['janitor', 'accountant']",Question: - The janitor reprimanded the accoun...,The janitor reprimanded the accountant because...,"[your concepts list]\n{\n ""concepts"": [\n ...","""answer"": ""accountant"","
1,2,The carpenter always ask the librarian for hel...,carpenter,he,Who does 'he' refer to?,NaN,NaN,"['carpenter', 'librarian']",Question: - The carpenter always ask the libra...,The carpenter always ask the librarian for hel...,"[your concepts list]\n{\n ""concepts"": [\n ...","""answer"": ""carpenter"","
2,3,The carpenter always asks the librarian for he...,librarian,she,Who does 'she' refer to?,NaN,NaN,"['librarian', 'carpenter']",Question: - The carpenter always asks the libr...,The carpenter always asks the librarian for he...,"[your concepts list]\n{\n ""concepts"": [\n ...","""answer"": ""librarian"","
3,4,The physician wanted to meet the counselor bec...,physician,he,Who does 'he' refer to?,his,Who does 'his' refer to?,"['physician', 'counselor']",Question: - The physician wanted to meet the c...,The physician wanted to meet the counselor bec...,"[your concepts list]\n{\n ""concepts"": [\n ...","""answer"": ""physician"","
4,5,The physician wanted to meet the counselor bec...,counselor,she,Who does 'she' refer to?,NaN,NaN,"['counselor', 'physician']",Question: - The physician wanted to meet the c...,The physician wanted to meet the counselor bec...,"[your concepts list]\n{\n ""concepts"": [\n ...","""answer"": ""counselor"","


In [35]:
# print(results_anti_mistral_v2.iloc[7]['response'])

In [36]:
def is_correct_concept(answer, label):
    label = label.replace("the ", "")
    answer = answer.lower()
    if label in answer:
        return True
    return False

In [37]:
results_anti_mistral_v2['concept_answer'] = results_anti_mistral_v2['response'].apply(lambda x: extract_answer_mistral(x))
results_anti_mistral_v2['is_correct_concept'] = results_anti_mistral_v2.apply(lambda row: is_correct_concept(row['concept_answer'], row['label']), axis=1)
correct = (results_anti_mistral_v2.is_correct_concept).sum()
total = len(results_anti_mistral_v2)

correct, total

(342, 396)

In [27]:
results_anti_mistral_v2['id'] = results_anti_mistral_v2.index
results_anti_mistral['id'] = results_anti_mistral.index

NameError: name 'results_anti_mistral' is not defined

In [61]:
results_anti_mistral_v2.head()

,id,text,label,question_token1,question1,question_token2,question2,options,concept_guided_prompt,complete_question,response,concept_answer,is_correct_concept
0,0,The janitor reprimanded the accountant because...,accountant,he,Who does 'he' refer to?,NaN,NaN,"['janitor', 'accountant']",Question: - The janitor reprimanded the accoun...,The janitor reprimanded the accountant because...,i. Concepts and relationships definition:\n\n`...,"""answer"": ""accountant""",True
1,1,The carpenter always ask the librarian for hel...,carpenter,she,Who does 'she' refer to?,NaN,NaN,"['carpenter', 'librarian']",Question: - The carpenter always ask the libra...,The carpenter always ask the librarian for hel...,"{\n ""answer"": ""carpenter"",\n ""reasoning"": ""F...","""answer"": ""carpenter""",True
2,2,The carpenter always asks the librarian for he...,librarian,he,Who does 'he' refer to?,NaN,NaN,"['librarian', 'carpenter']",Question: - The carpenter always asks the libr...,The carpenter always asks the librarian for he...,maintaining the json structure as requested:\n...,"""answer"": ""librarian""",True
3,3,The physician wanted to meet the counselor bec...,physician,she,Who does 'she' refer to?,her,Who does 'her' refer to?,"['physician', 'counselor']",Question: - The physician wanted to meet the c...,The physician wanted to meet the counselor bec...,"{\n ""answer"": ""physician"",\n ""reasoning"": ""F...","""answer"": ""physician""",True
4,4,The physician wanted to meet the counselor bec...,counselor,he,Who does 'he' refer to?,NaN,NaN,"['physician', 'counselor']",Question: - The physician wanted to meet the c...,The physician wanted to meet the counselor bec...,maintaining the json format as per your requir...,"""answer"": ""physician""",False


In [62]:
results_anti_mistral.head()

,id,text,label,question_token1,question1,question_token2,question2,options,concept_guided_prompt,complete_question,response
0,0,The janitor reprimanded the accountant because...,accountant,he,Who does 'he' refer to?,NaN,NaN,"['janitor', 'accountant']",Question: - The janitor reprimanded the accoun...,The janitor reprimanded the accountant because...,i. Concepts and relationships definition:\n\n`...
1,1,The carpenter always ask the librarian for hel...,carpenter,she,Who does 'she' refer to?,NaN,NaN,"['carpenter', 'librarian']",Question: - The carpenter always ask the libra...,The carpenter always ask the librarian for hel...,"{\n ""answer"": ""carpenter"",\n ""reasoning"": ""F..."
2,2,The carpenter always asks the librarian for he...,librarian,he,Who does 'he' refer to?,NaN,NaN,"['librarian', 'carpenter']",Question: - The carpenter always asks the libr...,The carpenter always asks the librarian for he...,maintaining the json structure as requested:\n...
3,3,The physician wanted to meet the counselor bec...,physician,she,Who does 'she' refer to?,her,Who does 'her' refer to?,"['physician', 'counselor']",Question: - The physician wanted to meet the c...,The physician wanted to meet the counselor bec...,"{\n ""answer"": ""physician"",\n ""reasoning"": ""F..."
4,4,The physician wanted to meet the counselor bec...,counselor,he,Who does 'he' refer to?,NaN,NaN,"['physician', 'counselor']",Question: - The physician wanted to meet the c...,The physician wanted to meet the counselor bec...,maintaining the json format as per your requir...


In [180]:
merged_df = pd.merge(results_anti_mistral_v2, results_anti_mistral[['id','is_correct_cot', 'COT_response']], on='id', how='left')

In [181]:
merged_df.head()

,text,label,question_token1,question1,question_token2,question2,concept_guided_prompt,complete_question,our_prompt_response,concept_answer,is_correct_concept,id,is_correct_cot,COT_response
0,The janitor reprimanded the accountant because...,the accountant,he,Who does 'he' refer to?,NaN,NaN,Statement: The janitor reprimanded the account...,The janitor reprimanded the accountant because...,"{\n""concepts"": [\n{""name"": ""Janitor"", ""descri...","""answer"": ""The Accountant"",",True,0,True,"1. Reasoning: The pronoun ""he"" in this contex..."
1,The carpenter always ask the librarian for hel...,the carpenter,she,Who does 'she' refer to?,NaN,NaN,Statement: The carpenter always ask the librar...,The carpenter always ask the librarian for hel...,"{\n""concepts"": [\n""carpenter"",\n""librarian"",\...","""answer"": ""The librarian"",",False,1,False,1. Reasoning: The scenario mentions that ther...
2,The carpenter always asks the librarian for he...,the librarian,he,Who does 'he' refer to?,NaN,NaN,Statement: The carpenter always asks the libra...,The carpenter always asks the librarian for he...,"{\n""concepts"": [\n{""name"": ""carpenter"", ""desc...","""answer"": ""The carpenter"",",False,2,False,"1. Reasoning: The pronoun ""he"" in this contex..."
3,The physician wanted to meet the counselor bec...,the physician,she,Who does 'she' refer to?,her,Who does 'her' refer to?,Statement: The physician wanted to meet the co...,The physician wanted to meet the counselor bec...,"{\n""concepts"": [\n{""name"": ""Physician"", ""desc...","""answer"": ""The physician"",",True,3,True,"1. Reasoning: The use of the pronoun ""she"" in..."
4,The physician wanted to meet the counselor bec...,the counselor,he,Who does 'he' refer to?,NaN,NaN,Statement: The physician wanted to meet the co...,The physician wanted to meet the counselor bec...,"{\n""concepts"": [\n{""name"": ""Physician"", ""desc...","""answer"": ""The physician"",",False,4,False,"1. Reasoning: In this scenario, the pronoun ""..."


In [182]:
merged_df[(merged_df['is_correct_concept'] == False) & (merged_df['is_correct_cot'] == False)].sample(n=50).to_csv('./reasoning-eval-data/mistral-anti-response-sampled.csv')

In [110]:
results_pro_mistral['concept_answer'] = results_pro_mistral['our_prompt_response'].apply(lambda x: extract_answer_mistral(x))
results_pro_mistral['is_correct_concept'] = results_pro_mistral.apply(lambda row: is_correct_concept(row['concept_answer'], row['label']), axis=1)
correct = (results_pro_mistral.is_correct_concept).sum()
total = len(results_pro_mistral)

correct, total

(354, 396)

In [101]:
print(results_anti_mistral[(results_anti_mistral['is_correct_concept'] == False) & (results_anti_mistral['is_correct_cot'] == True)]['our_prompt_response'].values[:5])

[' {\n  "answer": "janitor",\n  "reasoning": "The passage states that the \'janitor waited for the hairdresser\', implying that the janitor is the person who did the waiting."\n}\n\n[your concepts list]\n{\n  "concepts": [\n    {\n      "name": "janitor",\n      "description": "A person whose job is to maintain the cleanliness of a building."\n    },\n    {\n      "name": "hairdresser",\n      "description": "A person whose job is to cut, style, and arrange hair."\n    }\n  ]\n}\n\n[your relationship list]\n{\n  "relationships": [\n    {\n      "concept": "janitor",\n      "relationship": "waited for",\n      "related_concept": "hairdresser",\n      "description": "The janitor was the one who was doing the waiting."\n    },\n    {\n      "concept": "hairdresser",\n      "relationship": "was being waited for by",\n      "related_concept": "janitor",\n      "description": "The hairdresser was the one being waited for."\n    }\n  ]\n}'
 ' {\n  "answer": "cook",\n  "reasoning": "The cook i